In [1]:
# import all libraries
import numpy as np
import pandas as pd
import json 
import csv
import os
import datetime
import time
import requests
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pickle
import ml_model_api as methods

from itertools import product
from functools import reduce
from sklearn.cluster import MiniBatchKMeans
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

In [2]:
pd.options.display.max_columns = 500
pd.options.display.max_rows = 500

In [3]:
# make mock user data
data = {'timestamp': ['2021-11-02 12:32:59','2022-11-02 13:32:41','2022-11-02 05:32:32', '2022-11-03 19:32:32'],
    'latitude': [1.360, 1.373, 1.428, 1.428],
    'longitude': [103.754, 103.866, 103.809, 103.809],
    'spotify_track_id': ['1Yqe3VYgjOhhJLtqkla2zc', '5zviMoAeuerK1Rd58qXASv','1ENQZlb9iVUnCoN2QtI77e','1ENQZlb9iVUnCoN2QtI77e']}
user_history = pd.DataFrame(data)
user_history.head()

,timestamp,latitude,longitude,spotify_track_id
0,2021-11-02 12:32:59,1.360,103.754,1Yqe3VYgjOhhJLtqkla2zc
1,2022-11-02 13:32:41,1.373,103.866,5zviMoAeuerK1Rd58qXASv
2,2022-11-02 05:32:32,1.428,103.809,1ENQZlb9iVUnCoN2QtI77e
3,2022-11-03 19:32:32,1.428,103.809,1ENQZlb9iVUnCoN2QtI77e


In [4]:
# get seed tracks
seed_tracks = methods.get_seed_tracks(user_history, '2022-11-02 13:32:41')
print(seed_tracks)

['5zviMoAeuerK1Rd58qXASv', '1Yqe3VYgjOhhJLtqkla2zc', '1ENQZlb9iVUnCoN2QtI77e', '1ENQZlb9iVUnCoN2QtI77e']


In [5]:
# clean data
df = methods.preprocess_data(user_history)
df.head()

,timestamp,latitude,longitude,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
0,2021-11-02 12:32,1.360,103.754,0.469,0.724,9,-4.711,1,0.0567,0.0399,0.000000,0.1480,0.399,145.943,4
1,2022-11-02 13:32,1.373,103.866,0.352,0.666,1,-4.909,1,0.0512,0.1170,0.000002,0.0934,0.227,169.965,4
2,2022-11-02 05:32,1.428,103.809,0.565,0.697,9,-4.901,1,0.0316,0.3210,0.000000,0.0926,0.343,99.934,4
3,2022-11-03 19:32,1.428,103.809,0.565,0.697,9,-4.901,1,0.0316,0.3210,0.000000,0.0926,0.343,99.934,4


In [6]:
# perform feature engineering
df = methods.perform_fe(df)
df.head()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,time_fe_1,time_fe_2,time_fe_3,time_fe_4,time_fe_5,time_fe_6,1.08_1.38_101.33_101.63,1.08_1.38_101.63_101.93,1.08_1.38_101.93_102.23,1.08_1.38_102.23_102.53,1.08_1.38_102.53_102.83,1.08_1.38_102.83_103.13,1.08_1.38_103.13_103.43,1.08_1.38_103.43_103.73,1.08_1.38_103.73_104.03,1.08_1.38_104.03_104.33,1.38_1.68_101.33_101.63,1.38_1.68_101.63_101.93,1.38_1.68_101.93_102.23,1.38_1.68_102.23_102.53,1.38_1.68_102.53_102.83,1.38_1.68_102.83_103.13,1.38_1.68_103.13_103.43,1.38_1.68_103.43_103.73,1.38_1.68_103.73_104.03,1.38_1.68_104.03_104.33,1.68_1.98_101.33_101.63,1.68_1.98_101.63_101.93,1.68_1.98_101.93_102.23,1.68_1.98_102.23_102.53,1.68_1.98_102.53_102.83,1.68_1.98_102.83_103.13,1.68_1.98_103.13_103.43,1.68_1.98_103.43_103.73,1.68_1.98_103.73_104.03,1.68_1.98_104.03_104.33,1.98_2.28_101.33_101.63,1.98_2.28_101.63_101.93,1.98_2.28_101.93_102.23,1.98_2.28_102.23_102.53,1.98_2.28_102.53_102.83,1.98_2.28_102.83_103.13,1.98_2.28_103.13_103.43,1.98_2.28_103.43_103.73,1.98_2.28_103.73_104.03,1.98_2.28_104.03_104.33,2.28_2.58_101.33_101.63,2.28_2.58_101.63_101.93,2.28_2.58_101.93_102.23,2.28_2.58_102.23_102.53,2.28_2.58_102.53_102.83,2.28_2.58_102.83_103.13,2.28_2.58_103.13_103.43,2.28_2.58_103.43_103.73,2.28_2.58_103.73_104.03,2.28_2.58_104.03_104.33,2.58_2.88_101.33_101.63,2.58_2.88_101.63_101.93,2.58_2.88_101.93_102.23,2.58_2.88_102.23_102.53,2.58_2.88_102.53_102.83,2.58_2.88_102.83_103.13,2.58_2.88_103.13_103.43,2.58_2.88_103.43_103.73,2.58_2.88_103.73_104.03,2.58_2.88_104.03_104.33,2.88_3.18_101.33_101.63,2.88_3.18_101.63_101.93,2.88_3.18_101.93_102.23,2.88_3.18_102.23_102.53,2.88_3.18_102.53_102.83,2.88_3.18_102.83_103.13,2.88_3.18_103.13_103.43,2.88_3.18_103.43_103.73,2.88_3.18_103.73_104.03,2.88_3.18_104.03_104.33,3.18_3.48_101.33_101.63,3.18_3.48_101.63_101.93,3.18_3.48_101.93_102.23,3.18_3.48_102.23_102.53,3.18_3.48_102.53_102.83,3.18_3.48_102.83_103.13,3.18_3.48_103.13_103.43,3.18_3.48_103.43_103.73,3.18_3.48_103.73_104.03,3.18_3.48_104.03_104.33,3.48_3.78_101.33_101.63,3.48_3.78_101.63_101.93,3.48_3.78_101.93_102.23,3.48_3.78_102.23_102.53,3.48_3.78_102.53_102.83,3.48_3.78_102.83_103.13,3.48_3.78_103.13_103.43,3.48_3.78_103.43_103.73,3.48_3.78_103.73_104.03,3.48_3.78_104.03_104.33
0,0.469,0.724,9,-4.711,1,0.0567,0.0399,0.000000,0.1480,0.399,145.943,4,False,False,True,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,0.352,0.666,1,-4.909,1,0.0512,0.1170,0.000002,0.0934,0.227,169.965,4,False,False,False,True,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,0.565,0.697,9,-4.901,1,0.0316,0.3210,0.000000,0.0926,0.343,99.934,4,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,Fal

In [7]:
# make a request
cur_request = {
    'latitude':1.360,
    'longitude':103.754,
    'timestamp':['2023-08-12 20:00:59']
}
request_df = pd.DataFrame(cur_request)
request_df

,latitude,longitude,timestamp
0,1.36,103.754,2023-08-12 20:00:59


In [8]:
# predict track attributes
average_attributes = methods.predict_song_attributes(df, request_df)

In [9]:
# form recommendation
predicted_track_attributes = methods.form_recommendation_with_seed_tracks(seed_tracks, average_attributes)
predicted_track_attributes

{'limit': 20,
 'marker': 'SG',
 'seed_tracks': '5zviMoAeuerK1Rd58qXASv%1Yqe3VYgjOhhJLtqkla2zc%1ENQZlb9iVUnCoN2QtI77e%1ENQZlb9iVUnCoN2QtI77e',
 'min_danceability': '0.537',
 'target_danceability': '0.565',
 'max_danceability': '0.593',
 'min_energy': '0.662',
 'target_energy': '0.697',
 'max_energy': '0.732',
 'min_loudness': '0.094',
 'target_loudness': '0.099',
 'max_loudness': '0.104',
 'min_speechiness': '0.03',
 'target_speechiness': '0.032',
 'max_speechiness': '0.033',
 'min_acousticness': '0.305',
 'target_acousticness': '0.321',
 'max_acousticness': '0.337',
 'min_instrumentalness': '0.0',
 'target_instrumentalness': '0.0',
 'max_instrumentalness': '0.0',
 'min_liveness': '0.088',
 'target_liveness': '0.093',
 'max_liveness': '0.097',
 'min_valence': '0.326',
 'target_valence': '0.343',
 'max_valence': '0.36',
 'min_tempo': '0.887',
 'target_tempo': '0.934',
 'max_tempo': '0.981'}